In [31]:
import spacy
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from collections import OrderedDict
import random
import pickle

In [32]:
nlp1 = spacy.load("pl_document_ner/output/model-best")

c:\Users\1boze\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'pl_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.4.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [33]:
df =pd.read_csv("DATASET_NO_UNUSUAL_AND_INTERPOLSKA.csv")
df

,Unnamed: 0.1,Unnamed: 0,categories,text,text_full,main_category,label_mid,label_high
0,0,0,114 Rzeczowe,['czy wspólnota mieszkaniowa może podjąć uchwa...,czy wspólnota mieszkaniowa może podjąć uchwałę...,rzeczowe,rzeczowe,prawo cywilne
1,1,1,114 Rzeczowe,['czy wspólnota mieszkaniowa może odzyskać pom...,czy wspólnota mieszkaniowa może odzyskać pomie...,rzeczowe,rzeczowe,prawo cywilne
2,2,3,15 Prawo ochrony danych osobowych,['czy uprawnienie mieszkańców wspólnoty mieszk...,czy uprawnienie mieszkańców wspólnoty mieszkan...,prawo ochrony danych osobowych,prawo administracyjne materialne,prawo administracyjne
3,3,4,"20 Prawo budowlane, 114 Rzeczowe",['czy budowa windy dla osób niepełnosprawnych ...,czy budowa windy dla osób niepełnosprawnych mo...,prawo budowlane,prawo administracyjne materialne,prawo administracyjne
4,4,5,114 Rzeczowe,['jakie czynności powinna podjąć wspólnota mie...,jakie czynności powinna podjąć wspólnota miesz...,rzeczowe,rzeczowe,prawo cywilne
...,...,...,...,...,...,...,...,...
2669,2716,4030,77 Ustawa o prawach pacjenta i Rzeczniku Praw ...,['czy dokumentacja medyczna powinna zostać wyd...,czy dokumentacja medyczna powinna zostać wydan...,ustawa o prawach pacjenta i rzeczniku praw pac...,ustawa o prawach pacjenta i rzeczniku praw pac...,prawo medyczne
2670,2717,4031,"59 Ustawa Prawo farmaceutyczne, 197 Inne - pra...","['czy apteka może sprzedawać wodę mineralną?',...",czy apteka może sprzedawać wodę mineralną czy ...,ustawa prawo farmaceutyczne,ustawa prawo farmaceutyczne,prawo farmaceutyczne
2671,2718,4032,"15 Prawo ochrony danych osobowych, 69 Prawo ka...",['co w przedmiotowej sprawie może zrobić klien...,co w przedmiotowej sprawie może zrobić klientka,prawo ochrony danych osobowych,prawo administracyjne materialne,prawo administracyjne
2672,2719,4033,79 Ustawa o zawodach lekarza i lekarza dentyst...,['czy lekarz stażysta może przeprowadzać kwali...,czy lekarz stażysta może przeprowadzać kwalifi...,ustawa o zawodach lekarza i lekarza dentysty,ustawa o zawodach lekarza i lekarza dentysty,prawo medyczne


In [65]:
def get_top_legal_affairs(legalAffairs):
    acceptanceValue = legalAffairs[0][1]
    result = []
    for legalAffair in legalAffairs:
        if legalAffair[1] == acceptanceValue:
            result.append(legalAffair)
    return [k for (k,v) in result]

def get_random_classification():
    tags = ['administrative_law','civil_law','medical_law','tax_law','criminal_law','pharmaceutical_law','labor_law','international_law','constitutional_law']
    rand = random.randrange(0, 6, 1)
    return tags[rand]

def classifyText(text: str,all: bool):
    try:
        doc = nlp1(text)
    except:
        return []
    legalAffairs = OrderedDict()
    for token in doc:
        if token.ent_type_ != '':
            if token.ent_type_ not in legalAffairs:
                legalAffairs[token.ent_type_] = 0
            legalAffairs[token.ent_type_] += 1
    legalAffairs = OrderedDict(sorted(legalAffairs.items(),key = lambda key: key[1], reverse=True))
    if len(legalAffairs) == 0:
        return []
    result = get_top_legal_affairs(list(legalAffairs.items()))
    if all is True:
        result = list(legalAffairs.items())
    return result

def getWordsFromTextByLegalAffair(text: str, legalAffairs: list):
    if len(legalAffairs) == 0:
        return []
    try:
        doc = nlp1(text)
    except:
        return ""
    result = []
    for token in doc:
        if token.ent_type_ != '':
            if token.ent_type_ == legalAffairs[0]:
                result.append(token.text)
    return result

In [35]:
print(classifyText('czy wspólnota mieszkaniowa może podjąć uchwałę dotyczącą zakupu wodomierzy odczytywanych radiowo do lokali, pokrywając ich koszty?',True))

[('', 15), ('civil_law', 3)]


In [36]:
# civil, administrative, pharmaceutical, labor, medical, international, tax, constitutional, criminal, unknown
# [0, 0.1, 0.2, 0.3, 0, 0, 0, 0, 0, 0]
def sentence2vector(sentence: str, classifications: list):
    tags = ['civil_law','administrative_law','pharmaceutical_law','labor_law','medical_law','international_law','tax_law','constitutional_law','criminal_law','unknown']
    result = [0]*10
    if len(sentence) == 0:
        print('SENTENCE HAS 0 LENGTH')
        return result
    numberOfWordsInSentence = len(sentence.strip().split(" "))
    for classification in classifications:
        index = tags.index(classification[0])
        result[index] += classification[1]
    for index, law in enumerate(result):
        result[index] = result[index]/numberOfWordsInSentence
    return result

In [37]:
print(sentence2vector('czy lekarz może zostać zatrudniony w szpitalu na podstawie umowy o pracę na okres próbny, jeżeli była wcześniej w tym miejscu zatrudniona jako rezydent (do końca 2018 roku)?',[('civil_law', 2), ('medical_law', 2), ('administrative_law', 1)]))

[0.07142857142857142, 0.03571428571428571, 0.0, 0.0, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0]


In [67]:
correctClassifications = 0
randomCorrectClassification = 0
zeroLabelsClassifications = 0
vectorizedSentences = []
bestWordsFromSentences = []
labelTags = {'administrative_law':'prawo administracyjne','civil_law':'prawo cywilne','medical_law':'prawo medyczne','tax_law':'prawo podatkowe','criminal_law':'prawo karne','pharmaceutical_law':'prawo farmaceutyczne', 'labor_law':'prawo pracy','international_law':'prawo miädzynarodowe','constitutional_law':'prawo konstytucyjne'}
for i in range(len(df)):
    label= df['label_high'][i]
    text = df['text_full'][i]
    classified = []
    if text is None or text != text:
        continue
    labelTag = classifyText(text,False)
    if len(labelTag) > 0:
        for l in labelTag:
            classified.append(labelTags[l])
    print('{}/{} label: {}, classified: {} | CORRECT?: {}'.format(i,len(df),label,classified ,label in classified))
    if label in classified:
        correctClassifications += 1
    if len(classified) == 0:
        zeroLabelsClassifications +=1
    if label == labelTags[get_random_classification()]:
        randomCorrectClassification += 1
    # Vectorize sentence for kohonen
    labelTagTuples = classifyText(text,True)
    # if len(labelTagTuples) == 0:
    #     labelTagTuples = [('unknown',1)]
    vector = sentence2vector(text, labelTagTuples)
    vectorizedSentences.append(vector)
    bestWords = []
    if len(labelTag) > 0:
        bestWords = getWordsFromTextByLegalAffair(text, labelTag)
    bestWordsFromSentences.append(bestWords)
    
print('Overall accuracy: {}%'.format(correctClassifications/len(df)*100))
print('Textes with zero labels in classification: {}, percent of all: {}%'.format(zeroLabelsClassifications,zeroLabelsClassifications/len(df)*100))
print('Overall accuracy for random classification: {}%'.format(randomCorrectClassification/len(df)*100))
with open("data/vectorized_sentences", "wb") as fp:   #Pickling
    pickle.dump(vectorizedSentences, fp)
print('Saved vectors to file')
with open("data/best_words_for_sentences", "wb") as fp:   #Pickling
    pickle.dump(bestWordsFromSentences, fp)
print('Saved best words to file')

0/2674 label: prawo cywilne, classified: ['prawo cywilne'] | CORRECT?: True
1/2674 label: prawo cywilne, classified: ['prawo medyczne', 'prawo farmaceutyczne'] | CORRECT?: False
2/2674 label: prawo administracyjne, classified: [] | CORRECT?: False
3/2674 label: prawo administracyjne, classified: ['prawo administracyjne'] | CORRECT?: True
4/2674 label: prawo cywilne, classified: ['prawo cywilne'] | CORRECT?: True
5/2674 label: prawo cywilne, classified: ['prawo cywilne'] | CORRECT?: True
6/2674 label: prawo administracyjne, classified: ['prawo medyczne'] | CORRECT?: False
7/2674 label: prawo cywilne, classified: ['prawo cywilne'] | CORRECT?: True
8/2674 label: prawo cywilne, classified: ['prawo cywilne'] | CORRECT?: True
9/2674 label: prawo cywilne, classified: ['prawo cywilne'] | CORRECT?: True
10/2674 label: prawo cywilne, classified: ['prawo cywilne'] | CORRECT?: True
11/2674 label: prawo cywilne, classified: ['prawo pracy', 'prawo farmaceutyczne', 'prawo cywilne'] | CORRECT?: True
1

In [66]:
bestWordsFromSentences

[['uchwałę', 'dotyczącą', 'lokali'],
 ['pomieszczenie'],
 [],
 ['niepełnosprawnych'],
 ['założenie', 'uchwałą', 'remontowego'],
 ['członkowie', 'zakup', 'podjęcie', 'uchwały', 'większością'],
 ['jaką', 'powinny'],
 ['stwierdzenie', 'skład', 'wchodzi'],
 ['uchybienie',
  'formalne',
  'podejmowanie',
  'uchwały',
  'głosów',
  'podmiot',
  'zarząd',
  'zarządcę',
  'wspólnej'],
 ['podjęta',
  'większością',
  'głosów',
  'dalszego',
  'zbierania',
  'głosów',
  'uchwały',
  'głosu',
  'zbierania',
  'głosów',
  'uchwały',
  'zarząd',
  'uchwały',
  'zbierania',
  'głosów',
  'uchwałą'],
 ['lokali'],
 ['wprowadzić'],
 ['duża', 'lokali', 'uchwałę', 'balkonów'],
 [],
 ['wybudować'],
 ['głosu', 'uchwały', 'sumy', 'głosów', 'oddanych', 'głosów'],
 ['uchwały', 'instalacji'],
 [],
 ['właścicielka'],
 ['posiadająca',
  'kartę',
  'inwalidy',
  'niebędąca',
  'parkowania',
  'parkingowym',
  'niepełnosprawnych',
  'niebędąca',
  'posiadająca',
  'kartę',
  'inwalidy'],
 ['naprawy', 'komórki'],
 

In [75]:
vectorizedSentences

In [186]:
#open text file in read mode
text_file = open("data/criminal_doc.txt", "r")
 
#read whole file to a string
data = text_file.read()
 
#close file
text_file.close()
 

In [187]:
d= classifyText(data)
print(d)

OrderedDict([('medical_law', 136), ('civil_law', 112), ('criminal_law', 52), ('administrative_law', 44), ('labor_law', 39), ('pharmaceutical_law', 20), ('tax_law', 5), ('international_law', 1), ('constitutional_law', 1)])
['medical_law']


In [182]:
doc = nlp1(data)

In [183]:
spacy.displacy.render(doc, style="ent", jupyter=True)